<h1> <b> Scraping Data Using API ✍ </b> </h1>
<p> Excute with Google Colab </p>

##1. Iimport Library

In [ ]:
!pip install beautifulsoup4
!pip install xlsxwriter
import requests
import time
import random
import pandas as pd
from bs4 import BeautifulSoup
from google.colab import drive
drive.mount('/content/drive')

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 153.0/153.0 kB 1.4 MB/s eta 0:00:00
Mounted at /content/drive


##2. Get the url for each job

In [ ]:
# ========================= NUMBER OF PAGES TO SCRAPE ==========================
pageStart = 1
pageEnd = 352

# ========================= URL PATTERN FOR EACH PAGE ==========================
url = 'https://careerbuilder.vn/search-jobs'
headers = {
    "Accept": "application/json, text/javascript, */*; q=0.01",
    "Accept-Encoding": "gzip, deflate, br",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36 Edg/114.0.1823.43",
    "Content-Length": "84",
    "Content-Type": "application/x-www-form-urlencoded; charset=UTF-8",
    "Accept-Encoding": "gzip, deflate, br",
    "X-Requested-With": "XMLHttpRequest",
    "Connection": "keep-alive"
}
linkJob_completed = []
linkJob_error = []

# ========================== ITERATE OVER EACH PAGE ============================
for page in range(pageStart, pageEnd + 1):
    try:
        # Generate the payload for the current page
        if page <= 9:
            payload = {'dataOne': 'a:1:{s:4:"PAGE";s:1:"%d";}' % page}
        elif 10 <= page <= 99:
            payload = {'dataOne': 'a:1:{s:4:"PAGE";s:2:"%d";}' % page}
        else:
            payload = {'dataOne': 'a:1:{s:4:"PAGE";s:3:"%d";}' % page}

        # Make a request to the website
        response = requests.post(
            url=url, headers=headers, params=payload, files=payload)
        if response.status_code == 200:
            # Extract job links
            for record in response.json().get('data'):
                linkJob_completed.append(record.get('LINK_JOB'))
        print('Page %d - Scrape Data is completed' % page)

    except:
        linkJob_error.append('Page %d - Scrape Data is error' % page)
        print('Page %d - Scrape Data is error' % page)

df_link = pd.DataFrame(linkJob_completed)
df_link.to_excel('linkJob_total.xlsx')

##3. Extract information for each job

In [ ]:
for n in range(1, 2):
    df_link = pd.read_excel(
        f"/content/drive/MyDrive/Colab-Notebooks/careerbuilder-scarping-usingAPI-linkJob/linkJob{n}.xlsx")
    linkJob_completed = list(df_link['link'])

    # ============================ DECLARE VAR =================================
    header_job = {
        "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7",
        "Accept-Encoding": "gzip, deflate, br",
        "Cache-Control": "max-age=0",
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36 Edg/114.0.1823.43",
        "Accept-Encoding": "gzip, deflate, br",
        'Connection': 'keep-alive'
    }

    jobIndustries, jobLocation, jobType, jobTitle, jobSalary, jobLevel, jobExperience, jobUpdated, jobLink = [
    ], [], [], [], [], [], [], [], []
    companyType, companyName, companySize, companyUrl, = [], [], [], []
    linkError = []
    i = 1

    for link in linkJob_completed:
        response = requests.get(url=link, headers=header_job)
        if response.status_code == 200:
            soup = BeautifulSoup(response.text, 'html.parser')

            try:
                # ====================== EXTRACR JOB TITLE ======================
                jobTitle_check = soup.find('h1', class_='title').text

                jobDetailContent = soup.find(
                    'section', class_='job-detail-content')

                # ==================== EXTRACR JOB INDUSTRIES ===================
                jobSalary_check = jobDetailContent.find(
                    'i', class_='fa-usd').parent.next_sibling.next_sibling.text

                try:
                    jobDetailContent_li = jobDetailContent.find_all('li')
                    jobIndustries_check = []
                    jobIndustries_select = jobDetailContent_li[1].p.find_all(
                        'a')
                    if jobIndustries_select:
                        jobIndustries_check = [
                            elem.text.strip() for elem in jobIndustries_select] + jobIndustries_check
                        jobIndustries_check = " - ".join(jobIndustries_check)
                        jobIndustries.append(jobIndustries_check)
                    else:
                        jobIndustries_select = jobDetailContent_li[2].p.find_all(
                            'a')
                        jobIndustries_check = [
                            elem.text.strip() for elem in jobIndustries_select] + jobIndustries_check
                        jobIndustries_check = " - ".join(jobIndustries_check)
                        jobIndustries.append(jobIndustries_check)
                except:
                    jobIndustries.append("noData")

                # ===== EXTRACR JOB SALARY, EXPERIENCE, LEVEL, TYPE, UPDATED ====

                try:
                    jobExperience.append(jobDetailContent.find(
                        'i', class_='fa-briefcase').parent.next_sibling.next_sibling.text.strip())
                except:
                    jobExperience.append("noData")

                try:
                    jobLevel.append(jobDetailContent.find(
                        'i', class_='mdi-account').parent.next_sibling.next_sibling.text)
                except:
                    jobLevel.append("noData")

                try:
                    jobType.append(jobDetailContent.find(
                        'em', class_='mdi-briefcase-edit').parent.next_sibling.next_sibling.text)
                except:
                    jobType.append("noData")

                try:
                    jobUpdated.append(jobDetailContent.find(
                        'em', class_='mdi-update').parent.next_sibling.next_sibling.text)
                except:
                    jobUpdated.append("noData")

                # ============ EXTRACR JOB LOCATION, LINK, TITLE ===============
                try:
                    jobLocation.append(jobDetailContent.find('a').text)
                    jobTitle.append(jobTitle_check)
                    jobSalary.append(jobSalary_check)
                except:
                    jobLocation.append("noData")
                    jobTitle.append("noData")
                    jobSalary.append("noData")

                jobLink.append(link)

                # ==================== EXTRACR COMPANY INFO =====================
                try:
                    companyName_check = soup.find(
                        'a', class_='job-company-name').text
                    companyName.append(companyName_check)

                    companyLink = soup.find(
                        'a', class_='job-company-name')['href']
                    response2 = requests.post(
                        url=companyLink, headers=header_job)
                    if response2.status_code == 200:
                        soup2 = BeautifulSoup(response2.text, 'html.parser')

                        # Extract company size from the <span> element
                        try:
                            size = soup2.find(
                                'span', class_='mdi-account-supervisor')
                            companySize.append(
                                size.parent.text.split(':', 1)[-1].strip())
                        except:
                            companySize.append("noData")

                        # Extract company link from the <span> element
                        try:
                            link = soup2.find('span', class_='mdi-link')
                            companyUrl.append(
                                link.parent.text.split(':', 1)[-1].strip())
                        except:
                            companyUrl.append("noData")

                        # Extract company type from the <span> element
                        try:
                            type_company = soup2.find(
                                'span', class_='mdi-gavel')
                            companyType.append(
                                type_company.parent.text.split(':', 1)[-1].strip())
                        except:
                            companyType.append("noData")
                        print(f'Link {i} is completed')
                except:
                    companyName.append("CarrerBuilder's client")
                    companySize.append("noData")
                    companyUrl.append("noData")
                    companyType.append("noData")
                    print(f'Link {i} is completed')

            except:
                try:
                    # =================== EXTRACR JOB LOCATION ==================
                    jobLocation_check = soup.find('p', class_='list-workplace').text

                    # =================== EXTRACR JOB INDUSTRIE =================
                    jobDetailContent = soup.find('div', class_='content')
                    jobDetailContent_td = jobDetailContent.find_all(
                        'td', class_='content')
                    try:
                        jobIndustries_select = jobDetailContent_td[0]
                        jobIndustries_check = []
                        jobIndustries_check = [
                            elem.text.strip() for elem in jobIndustries_select] + jobIndustries_check
                        jobIndustries_check = " ".join(jobIndustries_check)
                        jobIndustries.append(jobIndustries_check)
                    except:
                        jobIndustries.append("noData")

                    # == EXTRACR JOB SALARY, EXPERIENCE, LEVEL, TYPE, UPDATED ===
                    try:
                        jobSalary.append(jobDetailContent_td[1].p.text)
                    except:
                        jobSalary.append("noData")
                    try:
                        jobType.append(jobDetailContent_td[2].p.text)
                    except:
                        jobType.append("noData")
                    try:
                        jobLevel.append(jobDetailContent_td[3].p.text)
                    except:
                        jobLevel.append("noData")
                    try:
                        jobExperience.append(
                            jobDetailContent_td[4].p.text.strip())
                    except:
                        jobExperience.append("noData")
                    try:
                        jobUpdated.append(jobDetailContent_td[6].p.text)
                    except:
                        jobUpdated.append("noData")

                    # ==================== EXTRACR JOB TITLE ====================
                    try:
                        jobTitle.append(soup.find('div', class_='title').text)
                        jobLocation.append(jobLocation_check)
                    except:
                        jobTitle.append("noData")
                        jobLocation.append("noData")

                    jobLink.append(link)

                    # ================== EXTRACR COMPANY INFO ===================
                    try:
                        companyName_check = soup.find(
                            'a', class_='company').text
                        companyName.append(companyName_check)

                        companyLink = soup.find('a', class_='company')['href']
                        response2 = requests.post(
                            url=companyLink, headers=header_job)
                        if response2.status_code == 200:
                            soup2 = BeautifulSoup(
                                response2.text, 'html.parser')
                            try:
                                # Extract company size from the <span> element
                                infoCompany = soup2.find(
                                    'div', class_='cp_basic_info_details')
                                infoCompany_li = infoCompany.find_all('li')

                                try:
                                    companyUrl.append(
                                        infoCompany_li[2].text.split(':', 1)[-1].strip())
                                except:
                                    companyUrl.append("noData")

                                infoCompany_li_2_span = infoCompany_li[1].find_all(
                                    'span', class_='gr')

                                try:
                                    companySize.append(
                                        infoCompany_li_2_span[0].text.split(':', 1)[-1].strip())
                                except:
                                    companySize.append("noData")
                                try:
                                    companyType.append(
                                        infoCompany_li_2_span[1].text.split(':', 1)[-1].strip())
                                except:
                                    companyType.append("noData")

                                print(f'Link {i} is completed')

                            except:
                                # Extract company size from the <span> element
                                try:
                                    size = soup2.find(
                                        'span', class_='mdi-account-supervisor')
                                    companySize.append(
                                        size.parent.text.split(':', 1)[-1].strip())
                                except:
                                    companySize.append("noData")

                                # Extract company link from the <span> element
                                try:
                                    link = soup2.find(
                                        'span', class_='mdi-link')
                                    companyUrl.append(
                                        link.parent.text.split(':', 1)[-1].strip())
                                except:
                                    companyUrl.append("noData")

                                # Extract company type from the <span> element
                                try:
                                    type_company = soup2.find(
                                        'span', class_='mdi-gavel')
                                    companyType.append(
                                        type_company.parent.text.split(':', 1)[-1].strip())
                                except:
                                    companyType.append("noData")
                                print(f'Link {i} is completed')

                    except:
                        companyName.append("CarrerBuilder's client")
                        companySize.append("noData")
                        companyUrl.append("noData")
                        companyType.append("noData")
                        print(f'Link {i} is completed')

                except:
                    if jobLocation is not None:
                        linkError.append(link)
                        print(f'Link {i} is ERROR - Not equal')
                    else:
                        jobSalary.append("ERROR - HTML format")
                        jobExperience.append("ERROR - HTML format")
                        jobLevel.append("ERROR - HTML format")
                        jobType.append("ERROR - HTML format")
                        jobUpdated.append("ERROR - HTML format")
                        jobTitle.append("ERROR - HTML format")
                        jobLocation.append("ERROR - HTML format")
                        jobIndustries.append("ERROR - HTML format")
                        companyName.append("ERROR - HTML format")
                        companySize.append("ERROR - HTML format")
                        companyUrl.append("ERROR - HTML format")
                        companyType.append("ERROR - HTML format")
                        jobLink.append(link)
                        print(f'Link {i} is ERROR - Format HTML')

        else:
            jobSalary.append("Link ERROR")
            jobExperience.append("Link ERROR")
            jobLevel.append("Link ERROR")
            jobType.append("Link ERROR")
            jobUpdated.append("Link ERROR")
            jobTitle.append("Link ERROR")
            jobLocation.append("Link ERROR")
            jobIndustries.append("Link ERROR")
            companyName.append("Link ERROR")
            companySize.append("Link ERROR")
            companyUrl.append("Link ERROR")
            companyType.append("Link ERROR")
            jobLink.append(link)
            print(f'Link {i} is ERROR - StatusCode <> 200')
        i = i + 1

    df = pd.DataFrame(list(zip(companyType, companyName, companySize, companyUrl, jobIndustries, jobLocation, jobType, jobTitle, jobSalary, jobLevel, jobExperience, jobUpdated, jobLink)),
                      columns=['companyType', 'companyName', 'companySize', 'companyLink', 'jobIndustries', 'jobLocation', 'jobType', 'jobTitle', 'jobSalary', 'jobLevel', 'jobExperience', 'jobUpdated', 'jobLink'])
    df.to_excel(
        f'/content/drive/MyDrive/Colab-Notebooks/careerbuilder-scarping-usingAPI-completed/crawlData_CarrerBuilder{n}.xlsx', engine='xlsxwriter')

    df2 = pd.DataFrame(linkError)
    df2.to_excel(
        f'/content/drive/MyDrive/Colab-Notebooks/careerbuilder-scarping-usingAPI-linkError/linkError-crawlData_CarrerBuilder{n}.xlsx', engine='xlsxwriter')

    print(f'File linkJob{n} is completed')

Link 1 is completed
Link 2 is completed
Link 3 is completed
Link 4 is completed
Link 5 is completed
Link 6 is completed
Link 7 is completed
Link 8 is completed
Link 9 is completed
Link 10 is completed
Link 11 is completed
Link 12 is completed
Link 13 is completed
Link 14 is completed
Link 15 is completed
Link 16 is completed
Link 17 is completed
Link 18 is completed
Link 19 is completed
Link 20 is completed
Link 21 is completed
Link 22 is completed
Link 23 is completed
Link 24 is completed
Link 25 is completed
Link 26 is completed
Link 27 is completed
Link 28 is completed
Link 29 is completed
Link 30 is completed
Link 31 is completed
Link 32 is completed
Link 33 is completed
Link 34 is completed
Link 35 is completed
Link 36 is completed
Link 37 is completed
Link 38 is completed
Link 39 is completed
Link 40 is completed
Link 41 is completed
Link 42 is completed
Link 43 is completed
Link 44 is completed
Link 45 is completed
Link 46 is completed
Link 47 is completed
Link 48 is completed
L

In [ ]:
linkJob_completed.clear()
companyType.clear()
companyName.clear()
companySize.clear()
companyUrl.clear()
jobIndustries.clear()
jobLocation.clear()
jobType.clear()
jobTitle.clear()
jobSalary.clear()
jobLevel.clear()
jobExperience.clear()
jobUpdated.clear()
jobLink.clear()